In [75]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import collections

# !pip install openrouteservice
# import openrouteservice

In [61]:
files_dire = '../data/curated/'
df = pd.read_csv(f'{files_dire}/processed_listing.csv').iloc[: , 1:]
df.head()

,bed,bath,car,type,address,suburb,code,url,loc_address,lat,lon,weekly_rent,list_date
0,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,350.0,2016-12-01
1,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,330.0,2016-12-01
2,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,330.0,2016-01-01
3,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,330.0,2015-11-01
4,3,1,3,House,"19 FRALLON CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Frallon Crescent, Karingal, Frankston, Melbour...",-38.136790,145.152331,420.0,2021-01-01


In [63]:
df['list_date'] = df['list_date'].apply(pd.to_datetime)

In [64]:
n_rows = len(df)

In [65]:
len(list(set(df['address'].tolist())))

82556

In [41]:
df2 = df.iloc[0:1,:]
df2 = df2.assign(list_history = '')
df2.head()

,bed,bath,car,type,address,suburb,code,url,loc_address,lat,lon,weekly_rent,list_date,list_history
0,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,350.0,2016-12-01,


In [66]:
n_property = 0

In [67]:
past_listing = []
list_date = df.iloc[0]['list_date']
list_price = df.iloc[0]['weekly_rent']

In [68]:
for i in tqdm(range(1, n_rows)):
    # print(i)
    # print(df.iloc[i]['address'])
    if df.iloc[i]['address'] == df2.iloc[n_property]['address']:
        # print(past_listing)
        # print(list_date)
        # print(df2.iloc[n_property]['list_date'])
        days = (df.iloc[i]['list_date'] - list_date)/ np.timedelta64(1, 'Y')
        list_date = df.iloc[i]['list_date']
        price = list_price - df.iloc[i]['weekly_rent']
        list_price = df.iloc[i]['weekly_rent']
        if days != 0:
            temp = [days, price]
            past_listing.append(temp)
    else:
        df2.at[n_property, 'list_history'] = past_listing
        n_property += 1
        df2.loc[n_property] = df.iloc[i]
        past_listing = []
df2.at[n_property, 'list_history'] = past_listing

100%|██████████| 303639/303639 [04:29<00:00, 1126.34it/s]


In [69]:
df2['list_count'] = df2['list_history'].str.len()

In [113]:
df2 = df2.rename(columns={'code': 'postcode'}, errors="raise")
df2.head()

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,lon,weekly_rent,list_date,list_history,list_count
0,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,350.0,2016-12-01,"[[-0.9171988473411501, 0.0], [-0.1670123274262...",2
1,3,1,3,House,"19 FRALLON CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Frallon Crescent, Karingal, Frankston, Melbour...",-38.136790,145.152331,420.0,2021-01-01,"[[4.498381212482118, -100.0], [-0.082137210209...",2
2,3,1,2,AUF,"1/14 PARER STREET, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,145.140240,220.0,2008-10-01,"[[-13.087195493405067, 250.0]]",1
3,3,1,2,AUF,"1/14 PARER ST, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,145.140240,320.0,2021-01-01,"[[10.086449413745662, -90.0], [-1.248485595186...",3
4,3,2,2,House,"1 YORK CT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"York Court, Karingal, Frankston, Melbourne, Ci...",-38.139321,145.157489,400.0,2021-01-01,"[[8.167176601846718, -170.0]]",1


In [114]:
df2.to_csv('../data/curated/processed_listing_combined.csv')

In [87]:
col_list = (df2['list_count'].values.tolist())

In [89]:
col_list.sort()

In [90]:
counter = collections.Counter(col_list)
print(counter)

Counter({1: 30322, 2: 19577, 3: 12567, 4: 7812, 5: 4750, 6: 2845, 7: 1702, 8: 1007, 0: 638, 9: 558, 10: 305, 11: 205, 12: 101, 13: 56, 14: 34, 15: 21, 16: 13, 17: 12, 19: 7, 20: 5, 18: 4, 21: 3, 23: 3, 29: 3, 30: 2, 34: 2, 24: 1, 26: 1, 27: 1, 32: 1, 42: 1})


In [111]:
postcode_df = pd.read_csv('../data/raw/abs/australian_postcodes.csv')
postcode_df = postcode_df.loc[postcode_df['state'] == 'VIC']
postcode_df['locality'] = postcode_df['locality'].str.title()
postcode_df = postcode_df[['postcode', 'locality', 'SA2_NAME_2016', 'SA2_MAINCODE_2016']]
postcode_df = postcode_df.rename(columns={'locality': 'suburb'}, errors="raise")
postcode_df.head()

,postcode,suburb,SA2_NAME_2016,SA2_MAINCODE_2016
6151,3000,Melbourne,Melbourne,206041122.0
6152,3001,Melbourne,Melbourne,206041122.0
6153,3002,East Melbourne,East Melbourne,206041119.0
6154,3003,West Melbourne,West Melbourne,206041127.0
6155,3004,Melbourne,Southbank,206041126.0


In [144]:
df3 = pd.merge(df2, postcode_df, on=['postcode', 'suburb'])
df3['SA2_MAINCODE_2016'] = df3['SA2_MAINCODE_2016'].astype(int)
df3 = df3.rename(columns={'SA2_MAINCODE_2016': 'SA2'}, errors="raise")
df3.head()

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,lon,weekly_rent,list_date,list_history,list_count,SA2_NAME_2016,SA2
0,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,350.0,2016-12-01,"[[-0.9171988473411501, 0.0], [-0.1670123274262...",2,Frankston,214011371
1,3,1,3,House,"19 FRALLON CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Frallon Crescent, Karingal, Frankston, Melbour...",-38.136790,145.152331,420.0,2021-01-01,"[[4.498381212482118, -100.0], [-0.082137210209...",2,Frankston,214011371
2,3,1,2,AUF,"1/14 PARER STREET, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,145.140240,220.0,2008-10-01,"[[-13.087195493405067, 250.0]]",1,Frankston,214011371
3,3,1,2,AUF,"1/14 PARER ST, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,145.140240,320.0,2021-01-01,"[[10.086449413745662, -90.0], [-1.248485595186...",3,Frankston,214011371
4,3,2,2,House,"1 YORK CT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"York Court, Karingal, Frankston, Melbourne, Ci...",-38.139321,145.157489,400.0,2021-01-01,"[[8.167176601846718, -170.0]]",1,Frankston,214011371


In [145]:
df3.dtypes

bed                       int64
bath                      int64
car                       int64
type                     object
address                  object
suburb                   object
postcode                  int64
url                      object
loc_address              object
lat                     float64
lon                     float64
weekly_rent             float64
list_date        datetime64[ns]
list_history             object
list_count                int64
SA2_NAME_2016            object
SA2                       int64
dtype: object

In [146]:
income_df = pd.read_csv(f'{files_dire}/abs/income_distribution.csv')
income_df = income_df[['SA2', 'Earners', 'Median age of earners', 'Median', 'Mean', 'Top 10%']]
income_df = income_df.replace(',','', regex=True)
cols = ['Earners', 'Median age of earners', 'Median', 'Mean', 'Top 10%']
income_df[cols] = income_df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
income_df = income_df.rename(columns={'Earners': 'income_earner', 'Median age of earners': 'income_median_age', 'Median': 'income_median', 'Mean': 'income_mean', 'Top 10%': 'income_top_10_pct'}, errors="raise")
income_df.head()

,SA2,income_earner,income_median_age,income_median,income_mean,income_top_10_pct
0,201011001,7989.0,42.0,53932.0,63668.0,28.7
1,201011002,7595.0,47.0,53688.0,77876.0,38.4
2,201011003,13598.0,43.0,50593.0,60367.0,29.3
3,201011004,12722.0,40.0,45828.0,50778.0,26.1
4,201011005,4249.0,46.0,52377.0,63258.0,29.3


In [147]:
df3 = pd.merge(df3, income_df, on='SA2')
df3.head()

In [230]:
population_df = pd.read_csv(f'{files_dire}/abs/population.csv')
population_df = population_df.iloc[: , 8:]
col_list = ['0-4', '5-9', '10–14']
# 0-14 years (children), 15-24 years (early working age), 25-54 years (prime working age), 55-64 years (mature working age), 65 years and over (elderly)
population_df['population_children'] = population_df[['0-4', '5–9', '10–14']].sum(axis = 1)
population_df = population_df.drop(['0-4', '5–9', '10–14'], axis=1)
population_df['population_prime_working'] = population_df[['25–29', '30–34', '35–39', '40–44', '45–49', '50–54']].sum(axis = 1)
population_df = population_df.drop(['25–29', '30–34', '35–39', '40–44', '45–49', '50–54'], axis=1)
population_df['population_elderly'] = population_df[['65–69', '70–74', '75–79', '80–84', '85 and over']].sum(axis = 1)
population_df = population_df.drop(['65–69', '70–74', '75–79', '80–84', '85 and over'], axis=1)
population_df['population_early_working'] = population_df.iloc[:,2] + population_df.iloc[:,3]
population_df['population_mature_working'] = population_df.iloc[:,4] + population_df.iloc[:,5]
population_df = population_df.drop(['15–19', '20–24', '55–59', '60–64', 'SA2 name'], axis=1)
population_df = population_df.rename(columns={'Total persons': 'population_total', 'SA2 code': 'SA2'}, errors="raise")
population_df.head()

,SA2,population_total,population_children,population_prime_working,population_elderly,population_early_working,population_mature_working
0,201011001,16823,4075,6702,2098,2391,1557
1,201011002,12076,1806,4321,2750,1565,1634
2,201011005,7232,1532,2483,1295,944,978
3,201011006,10640,2299,4347,1543,1434,1017
4,201011007,4213,904,1533,594,559,623


In [231]:
df3 = pd.merge(df3, population_df, on='SA2')
df3.head()

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,...,population_prime_working_x,population_elderly_x,population_early_working_x,population_mature_working_x,population_total_y,population_children_y,population_prime_working_y,population_elderly_y,population_early_working_y,population_mature_working_y
0,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,...,10224,4091,2836,2861,23908,3896,10224,4091,2836,2861
1,3,1,3,House,"19 FRALLON CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Frallon Crescent, Karingal, Frankston, Melbour...",-38.136790,...,10224,4091,2836,2861,23908,3896,10224,4091,2836,2861
2,3,1,2,AUF,"1/14 PARER STREET, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,...,10224,4091,2836,2861,23908,3896,10224,4091,2836,2861
3,3,1,2,AUF,"1/14 PARER ST, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,...,10224,4091,2836,2861,23908,3896,10224,4091,2836,2861
4,3,2,2,House,"1 YORK CT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"York Court, Karingal, Frankston, Melbourne, Ci...",-38.139321,...,10224,4091,2836,2861,23908,3896,10224,4091,2836,2861


In [260]:
school_df = pd.read_csv('../data/raw/schools.csv')
school_df = school_df.drop(['Entity_Type', 'School_No', 'School_Status', 'Address_Line_1', 'Address_Line_2',
         'Address_Town', 'Address_State', 'Address_Postcode',
         'Postal_Address_Line_1', 'Postal_Address_Line_2', 'Postal_Town',
         'Postal_State', 'Postal_Postcode', 'Full_Phone_No', 'LGA_ID',
         'LGA_Name'], axis=1)
school_type = ['Primary', 'Secondary', 'Pri/Sec']
school_df = school_df.loc[school_df['School_Type'].isin(school_type)]
school_df['geo_coordinate'] = [(x, y) for x,y in zip(school_df['Y'], school_df['X'])]
school_df.head()

,Education_Sector,School_Name,School_Type,X,Y,geo_coordinate
0,Government,Alberton Primary School,Primary,146.666601,-38.617713,"(-38.617713, 146.666601)"
1,Government,Allansford and District Primary School,Primary,142.590393,-38.386281,"(-38.386281, 142.590393)"
2,Government,Avoca Primary School,Primary,143.475649,-37.084502,"(-37.084502, 143.475649)"
3,Government,Avenel Primary School,Primary,145.234722,-36.901368,"(-36.901368, 145.234722)"
4,Government,Warrandyte Primary School,Primary,145.213980,-37.742675,"(-37.742675, 145.21398)"


In [256]:
from scipy.spatial.distance import cdist
df3['geo_coordinate'] = [(x, y) for x,y in zip(df3['lat'], df3['lon'])]
df3.head()

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,...,population_elderly_x,population_early_working_x,population_mature_working_x,population_total_y,population_children_y,population_prime_working_y,population_elderly_y,population_early_working_y,population_mature_working_y,geo_coordinate
0,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,...,4091,2836,2861,23908,3896,10224,4091,2836,2861,"(-38.1463748, 145.1664942)"
1,3,1,3,House,"19 FRALLON CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Frallon Crescent, Karingal, Frankston, Melbour...",-38.136790,...,4091,2836,2861,23908,3896,10224,4091,2836,2861,"(-38.1367897, 145.1523308)"
2,3,1,2,AUF,"1/14 PARER STREET, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,...,4091,2836,2861,23908,3896,10224,4091,2836,2861,"(-38.143728, 145.14024)"
3,3,1,2,AUF,"1/14 PARER ST, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,...,4091,2836,2861,23908,3896,10224,4091,2836,2861,"(-38.143728, 145.14024)"
4,3,2,2,House,"1 YORK CT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"York Court, Karingal, Frankston, Melbourne, Ci...",-38.139321,...,4091,2836,2861,23908,3896,10224,4091,2836,2861,"(-38.1393209, 145.1574891)"


In [263]:
def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

def match_value(df, col1, x, col2):
    """ Match value x from col1 row to value in col2. """
    return df[df[col1] == x][col2].values[0]

In [298]:
df3['closest_school_loc'] = tqdm([closest_point(x, list(school_df['geo_coordinate'])) for x in df3['geo_coordinate']])
df3['school_name'] = tqdm([match_value(school_df, 'geo_coordinate', x, 'School_Name') for x in df3['closest_school_loc']])

100%|██████████| 61959/61959 [00:00<00:00, 5131607.79it/s]


In [299]:
df3

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,...,population_mature_working_x,population_total_y,population_children_y,population_prime_working_y,population_elderly_y,population_early_working_y,population_mature_working_y,geo_coordinate,closest_school_loc,school_name
0,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,...,2861,23908,3896,10224,4091,2836,2861,"(-38.1463748, 145.1664942)","(-38.141476, 145.166813)",Karingal Heights Primary School
1,3,1,3,House,"19 FRALLON CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Frallon Crescent, Karingal, Frankston, Melbour...",-38.136790,...,2861,23908,3896,10224,4091,2836,2861,"(-38.1367897, 145.1523308)","(-38.137924, 145.152277)",Karingal Primary School
2,3,1,2,AUF,"1/14 PARER STREET, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,...,2861,23908,3896,10224,4091,2836,2861,"(-38.143728, 145.14024)","(-38.14663, 145.145528)",Frankston East Primary School
3,3,1,2,AUF,"1/14 PARER ST, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,...,2861,23908,3896,10224,4091,2836,2861,"(-38.143728, 145.14024)","(-38.14663, 145.145528)",Frankston East Primary School
4,3,2,2,House,"1 YORK CT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"York Court, Karingal, Frankston, Melbourne, Ci...",-38.139321,...,2861,23908,3896,10224,4091,2836,2861,"(-38.1393209, 145.1574891)","(-38.137924, 145.152277)",Karingal Primary School
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61954,3,2,2,House,"7 EILEEN CLOSE, NORTH WARRANDYTE",North Warrandyte,3113,https://www.oldlistings.com.au/real-estate/VIC...,"Eileen Close, North Warrandyte, Melbourne, Shi...",-37.720603,...,1102,6985,1338,2502,1066,977,1102,"(-37.7206034, 145.2173285)","(-37.742675, 145.21398)",Warrandyte Primary School
61955,3,2,2,House,"187 RESEARCH-WARRANDYTE RD, NORTH WARRANDYTE",North Warrandyte,3113,https://www.oldlistings.com.au/real-estate/VIC...,"Research - Warrandyte Road, Sloan Hill, North ...",-37.726075,...,1102,6985,1338,2502,1066,977,1102,"(-37.7260753, 145.2110538)","(-37.742675, 145.21398)",Warrandyte Primary School
61956,4,2,2,House,"81 KANGAROO GROUND-WARRANDYTE ROAD, NORTH WARR...",North Warrandyte,3113,https://www.oldlistings.com.au/real-estate/VIC...,"Kangaroo Ground - Warrandyte Road, Sloan Hill,...",-37.734741,...,1102,6985,1338,2502,1066,977,1102,"(-37.7347411, 145.2227867)","(-37.742675, 145.21398)",Warrandyte Primary School
61957,3,1,1,House,"25 GREGORY STREET, OUYEN",Ouyen,3490,https://www.oldlistings.com.au/real-estate/VIC...,"Gregory Street, Ouyen, Rural City of Mildura, ...",-35.072039,...,609,3709,641,1254,856,349,609,"(-35.0720389, 142.3160326)","(-35.064571, 142.322608)",Ouyen P-12 College


In [300]:
population_projection_df = pd.read_csv('../data/raw/population_projection.csv')
population_projection_df = population_projection_df.loc[population_projection_df['SEX'] == 'Persons']
population_projection_df = population_projection_df.loc[population_projection_df['YEAR'] == 2027]
population_projection_df = population_projection_df.drop(['YEAR', 'SA2_NAME', 'SEX'], axis=1)

# 0-14 years (children), 15-24 years (early working age), 25-54 years (prime working age), 55-64 years (mature working age), 65 years and over (elderly)
population_projection_df['proj_population_children'] = population_projection_df[['Age0-4', 'Age5-9', 'Age10-14']].sum(axis = 1)
population_projection_df = population_projection_df.drop(['Age0-4', 'Age5-9', 'Age10-14'], axis=1)
population_projection_df['proj_population_early_working'] = population_projection_df[['Age15-19', 'Age20-24']].sum(axis = 1)
population_projection_df = population_projection_df.drop(['Age15-19', 'Age20-24'], axis=1)
population_projection_df['proj_population_prime_working'] = population_projection_df[['Age25-29', 'Age30-34', 'Age35-39', 'Age40-44', 'Age45-49', 'Age50-54']].sum(axis = 1)
population_projection_df = population_projection_df.drop(['Age25-29', 'Age30-34', 'Age35-39', 'Age40-44', 'Age45-49', 'Age50-54'], axis=1)
population_projection_df['proj_population_mature_working'] = population_projection_df[['Age55-59', 'Age60-64']].sum(axis = 1)
population_projection_df = population_projection_df.drop(['Age55-59', 'Age60-64'], axis=1)
population_projection_df['proj_population_elderly'] = population_projection_df[['Age65-69', 'Age70-74', 'Age75-79', 'Age80-84', 'Age85+']].sum(axis = 1)
population_projection_df = population_projection_df.drop(['Age65-69', 'Age70-74', 'Age75-79', 'Age80-84', 'Age85+'], axis=1)
population_projection_df = population_projection_df.rename(columns={'Total': 'proj_population_total', 'SA2_CODE': 'SA2'}, errors="raise")
population_projection_df.head()

,SA2,proj_population_total,proj_population_children,proj_population_early_working,proj_population_prime_working,proj_population_mature_working,proj_population_elderly
13862,201011001,18611,4389,2496,7275,1807,2644
13865,201011002,12252,1788,1796,3940,1674,3054
13868,201011003,26630,5094,2777,9809,3270,5680
13871,201011004,28423,4849,3734,11347,3056,5437
13874,201011005,8900,1919,1368,2966,1067,1580


In [301]:
df3 = pd.merge(df3, population_projection_df, on='SA2')
df3.head()

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,...,population_mature_working_y,geo_coordinate,closest_school_loc,school_name,proj_population_total,proj_population_children,proj_population_early_working,proj_population_prime_working,proj_population_mature_working,proj_population_elderly
0,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,...,2861,"(-38.1463748, 145.1664942)","(-38.141476, 145.166813)",Karingal Heights Primary School,26093,4247,3044,10817,3023,4962
1,3,1,3,House,"19 FRALLON CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Frallon Crescent, Karingal, Frankston, Melbour...",-38.136790,...,2861,"(-38.1367897, 145.1523308)","(-38.137924, 145.152277)",Karingal Primary School,26093,4247,3044,10817,3023,4962
2,3,1,2,AUF,"1/14 PARER STREET, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,...,2861,"(-38.143728, 145.14024)","(-38.14663, 145.145528)",Frankston East Primary School,26093,4247,3044,10817,3023,4962
3,3,1,2,AUF,"1/14 PARER ST, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,...,2861,"(-38.143728, 145.14024)","(-38.14663, 145.145528)",Frankston East Primary School,26093,4247,3044,10817,3023,4962
4,3,2,2,House,"1 YORK CT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"York Court, Karingal, Frankston, Melbourne, Ci...",-38.139321,...,2861,"(-38.1393209, 145.1574891)","(-38.137924, 145.152277)",Karingal Primary School,26093,4247,3044,10817,3023,4962


In [302]:
population_projection_df.columns

Index(['SA2', 'proj_population_total', 'proj_population_children',
       'proj_population_early_working', 'proj_population_prime_working',
       'proj_population_mature_working', 'proj_population_elderly'],
      dtype='object')

In [303]:
df3.to_csv('../data/curated/listing_with_features.csv')